# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Create Table Statement #1

In [11]:
query = "CREATE TABLE IF NOT EXISTS sessions_table"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

### Insert Data Into Table #1

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:   
        query = "INSERT INTO sessions_table (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int (line[8]), int(line[3]), str(line[0]), str(line[-2]), float (line[5])))

### Select to Verify Data

This query provides the artist, song title and song length in the music app history that was heard during sessionID = 338 and itemInSession = 4. Since we need to slice data by sessionID and itemInSession, I set the primary key and itemInSession since we will be partitioning the data by those columns. 

In [13]:
select = "SELECT artist, song, length FROM sessions_table WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Create Table Statement #2

In [11]:
query = "CREATE TABLE IF NOT EXISTS users_table"
query = query + """(userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int, 
        PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)



### Insert Data Into Table #2

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:   
        query = "INSERT INTO users_table (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int (line[-1]), int(line[-3]), str(line[0]), str(line[-2]), str (line[1]), str (line[4]), int (line[3])))

### Select to Verify Data

This query selects the artist name, and user (first and last name) for userid=10 and sessionId = 182. The query also sorts the results by itemInSession. Since we are using userId, and sessionId to identify our data criteria, I am using those metrics to partition the data. I am also using itemInSession as the composite column since we want to sort by this metric. 

In [13]:
select = "SELECT artist, song, firstName, lastName FROM users_table WHERE sessionId = 182 AND userId = 10"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Example of Invalid Query 

In [14]:
select = "SELECT firstName, lastName FROM users_table WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


### Create Table Statement #3

In [22]:
query = "CREATE TABLE IF NOT EXISTS songs_table"
query = query + """(firstName text, lastName text, song text, userId int, PRIMARY KEY (song, userId)) """
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### Insert Data Into Table #3

In [25]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:   
        query = "INSERT INTO songs_table (firstName, lastName, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (str (line[1]), str(line[4]), str(line[-2]), int (line[-1])))

### Select to Verify Data

This query selects all user names (first and last) in my dataset who listened to the song "All Hands Against Hist Own." Since we want to slice our data by song, I am partitioning the data by song. However, since there may be more than one user who listens to a song, and in order to create a unique primary key(song-user combination), I am also partitioning the data by userId 

In [26]:
select = "SELECT firstName, lastName FROM songs_table WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
query = ["drop table if exists songs_table", "drop table if exists users_table", "drop table if exists sessions_table" ]

try:
    for  each in query:
        rows = session.execute(each)
        break
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()